## 라이브러리

In [1]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## 사전작업

### 데이터 불러오기

In [2]:
df=pd.read_csv('./data/Kurly.csv',encoding='utf-8')

### 전처리

In [3]:
def ingredient_to_list(text):
    text=text[1:-2]
    text=re.sub("'",'',text)
    text=text.replace(' ','')
    return text

df['ingredient']=df['이미지성분'].apply(ingredient_to_list)
df.drop(['이미지성분'],axis=1,inplace=True)

df=df[df['ingredient']!='']
df=df.reset_index(drop=True)
df

,category,brand,item,allergy,ingredient
0,국/반찬/메인 요리,풀무원,국산콩 나또 3종,"국산콩생나또,대두,밀,쇠고기,검은콩생나또,대두,밀,쇠고기,국산콩와사비나또,대두,밀,...","고등어,아황산류,메밀,옥수수유옥수수,복숭아,오징어,홍합포함,설탕,조개류,전복,대두,..."
1,국/반찬/메인 요리,kurly,전통 시장 물떡 (2팩),"밀,대두,우유,계란,우유,메밀,땅콩,고등어,게,새우,돼지고기,복숭아,토마토,아황산류...","조개류,정제소금,주정,떡류,조기,소스소스,참치,비타민,가공소금,오징어,구연산,복합조..."
2,국/반찬/메인 요리,My little recipe,안동에서 올라온 전통찜닭 대용량 (3~4인용),"닭고기,오징어,밀,대두,닭고기,오징어,우유,대두,밀",정제수
3,국/반찬/메인 요리,누테이블,메추리알 장조림 350g,"알류(메추리알),대두,밀,조개류(굴),알류(계란),우유,메밀,땅콩,고등어,게,새우,...","천일염,양조간장"
4,국/반찬/메인 요리,모두의맛집,소문난 원조 조방낙지 낙곱새,"대두,밀,새우,쇠고기,조개류(굴),난류,우유,메밀,땅콩,고등어,게,돼지고기,복숭아,...","마늘,조개류,전복,내두유,콩기름,홍합포함"
...,...,...,...,...,...
2925,간식/과자/떡,이브뚜리에,봉봉 초콜렛 12P,"우유,대두,밀","아몬드,천연향료,유청,전화당시럼,해이즐,전분,코코아파우더,글리셰린,프랄린,구연산,전..."
2926,간식/과자/떡,청우,아홉 번 구워만든 죽염캔디 38g,"땅콩,대두,우유,복숭아","무엇,자일리톨,박하유"
2927,간식/과자/떡,타르투프랑게,트뤼플랏 어소티드 스위트 초콜렛 70g (팩),"밀,우유,대두,보리,땅콩,계란","아몬드,크런치캐러멜,준초콜릿,포도당시련,해이즐,화이트초콜릿,코코아애스,유지방문일유당..."
2928,간식/과자/떡,kurly,맛다시마 젤리 100g (냉장),없음,"설탕,소금,대두,매실과육,다시매,간장"


In [4]:
for idx,text in enumerate(df['allergy']):
    text=text.replace('혼입가능','')
    text=text.replace('(',',')
    text=text.replace(')',',')
    text=re.sub(',+',',',text)
    df['allergy'][idx]=text
df

,category,brand,item,allergy,ingredient
0,국/반찬/메인 요리,풀무원,국산콩 나또 3종,"국산콩생나또,대두,밀,쇠고기,검은콩생나또,대두,밀,쇠고기,국산콩와사비나또,대두,밀,...","고등어,아황산류,메밀,옥수수유옥수수,복숭아,오징어,홍합포함,설탕,조개류,전복,대두,..."
1,국/반찬/메인 요리,kurly,전통 시장 물떡 (2팩),"밀,대두,우유,계란,우유,메밀,땅콩,고등어,게,새우,돼지고기,복숭아,토마토,아황산류...","조개류,정제소금,주정,떡류,조기,소스소스,참치,비타민,가공소금,오징어,구연산,복합조..."
2,국/반찬/메인 요리,My little recipe,안동에서 올라온 전통찜닭 대용량 (3~4인용),"닭고기,오징어,밀,대두,닭고기,오징어,우유,대두,밀",정제수
3,국/반찬/메인 요리,누테이블,메추리알 장조림 350g,"알류,메추리알,대두,밀,조개류,굴,알류,계란,우유,메밀,땅콩,고등어,게,새우,돼지고...","천일염,양조간장"
4,국/반찬/메인 요리,모두의맛집,소문난 원조 조방낙지 낙곱새,"대두,밀,새우,쇠고기,조개류,굴,난류,우유,메밀,땅콩,고등어,게,돼지고기,복숭아,토...","마늘,조개류,전복,내두유,콩기름,홍합포함"
...,...,...,...,...,...
2925,간식/과자/떡,이브뚜리에,봉봉 초콜렛 12P,"우유,대두,밀","아몬드,천연향료,유청,전화당시럼,해이즐,전분,코코아파우더,글리셰린,프랄린,구연산,전..."
2926,간식/과자/떡,청우,아홉 번 구워만든 죽염캔디 38g,"땅콩,대두,우유,복숭아","무엇,자일리톨,박하유"
2927,간식/과자/떡,타르투프랑게,트뤼플랏 어소티드 스위트 초콜렛 70g (팩),"밀,우유,대두,보리,땅콩,계란","아몬드,크런치캐러멜,준초콜릿,포도당시련,해이즐,화이트초콜릿,코코아애스,유지방문일유당..."
2928,간식/과자/떡,kurly,맛다시마 젤리 100g (냉장),없음,"설탕,소금,대두,매실과육,다시매,간장"


### sample 데이터 추출

In [5]:
sample_index=df.sample(5,random_state=1111).index.values
sample_data=df.iloc[sample_index].reset_index(drop=True)
train_index=list(set(df.index.values)-set(sample_index))
train_data=df.iloc[train_index].reset_index(drop=True)

## 분류

### 분류 기준표

In [6]:
kurly_ingredient=pd.read_csv('./data/컬리 성분 정리표_final.csv',encoding='cp949')

kurly_milk=kurly_ingredient.loc[kurly_ingredient['분류']=='유제품','성분'].tolist()
kurly_sea=kurly_ingredient.loc[kurly_ingredient['분류']=='해산물','성분'].tolist()
kurly_meat=kurly_ingredient.loc[kurly_ingredient['분류']=='육류','성분'].tolist()
kurly_egg=kurly_ingredient.loc[kurly_ingredient['분류']=='달걀','성분'].tolist()
kurly_honey=kurly_ingredient.loc[kurly_ingredient['분류']=='꿀','성분'].tolist()

In [7]:
# 육류 리스트
with open("./list/meat.txt", "r") as file:
    lst = file.readlines()

for word in lst:
    word=word.replace('\n','')
    if len(word)>0:
        kurly_meat.append(word)

meat_lst=[]
for word in kurly_meat:
    word=word.replace(' ','')
    meat_lst.append(word)

meat_lst=list(set(meat_lst))

In [8]:
# 해산물 리스트
with open("./list/sea.txt", "r") as file:
    lst = file.readlines()

for word in lst:
    word=word.replace('\n','')
    if len(word)>0:
        kurly_sea.append(word)

sea_lst=[]
for word in kurly_sea:
    word=word.replace(' ','')
    sea_lst.append(word)

sea_lst=list(set(sea_lst))

In [9]:
# 유제품 리스트
with open("./list/milk.txt", "r") as file:
    lst = file.readlines()

for word in lst:
    word=word.replace('\n','')
    if len(word)>0:
        kurly_milk.append(word)

milk_lst=[]
for word in kurly_milk:
    word=word.replace(' ','')
    milk_lst.append(word)

milk_lst=list(set(milk_lst))

In [10]:
# 달걀 리스트
with open("./list/egg.txt", "r") as file:
    lst = file.readlines()

for word in lst:
    word=word.replace('\n','')
    if len(word)>0:
        kurly_egg.append(word)

egg_lst=[]
for word in kurly_egg:
    word=word.replace(' ','')
    egg_lst.append(word)

egg_lst=list(set(egg_lst))

In [11]:
honey_lst=[]
for word in kurly_honey:
    word=word.replace(' ','')
    honey_lst.append(word)

honey_lst=list(set(honey_lst))
honey_lst.append('그래놀라')

### 비건 단계별 분류

In [12]:
train_data['classify']='0'
for i in range(len(train_data)):
    flag=0
    milk=0
    egg=0
    for word in train_data['ingredient'][i].split(','):
        if word in meat_lst:
            flag=1
            break
    if flag==0:
        for word in train_data['ingredient'][i].split(','):
            if word in sea_lst:
                flag=2
                break
    if flag==0:
        for word in train_data['ingredient'][i].split(','):
            if word in milk_lst:
                flag=3
                milk+=1
                break
        for word in train_data['ingredient'][i].split(','):
            if word in egg_lst:
                flag=3
                egg+=1
                break
    if flag==0:
        for word in train_data['ingredient'][i].split(','):
            if word in honey_lst:
                flag=4
                break
            
    if flag==1:
        classify='general'
    elif flag==2:
        classify='pesco'
    elif flag==3:
        if milk>0 and egg==0:
            classify='lacto'
        elif milk==0 and egg>0:
            classify='ovo'
        else:
            classify='lacto ovo'
    elif flag==4:
        classify='honey'
    else:
        classify='vegan'
    train_data['classify'][i]=classify

In [13]:
train_data

,category,brand,item,allergy,ingredient,classify
0,국/반찬/메인 요리,풀무원,국산콩 나또 3종,"국산콩생나또,대두,밀,쇠고기,검은콩생나또,대두,밀,쇠고기,국산콩와사비나또,대두,밀,...","고등어,아황산류,메밀,옥수수유옥수수,복숭아,오징어,홍합포함,설탕,조개류,전복,대두,...",general
1,국/반찬/메인 요리,kurly,전통 시장 물떡 (2팩),"밀,대두,우유,계란,우유,메밀,땅콩,고등어,게,새우,돼지고기,복숭아,토마토,아황산류...","조개류,정제소금,주정,떡류,조기,소스소스,참치,비타민,가공소금,오징어,구연산,복합조...",pesco
2,국/반찬/메인 요리,My little recipe,안동에서 올라온 전통찜닭 대용량 (3~4인용),"닭고기,오징어,밀,대두,닭고기,오징어,우유,대두,밀",정제수,vegan
3,국/반찬/메인 요리,누테이블,메추리알 장조림 350g,"알류,메추리알,대두,밀,조개류,굴,알류,계란,우유,메밀,땅콩,고등어,게,새우,돼지고...","천일염,양조간장",vegan
4,국/반찬/메인 요리,모두의맛집,소문난 원조 조방낙지 낙곱새,"대두,밀,새우,쇠고기,조개류,굴,난류,우유,메밀,땅콩,고등어,게,돼지고기,복숭아,토...","마늘,조개류,전복,내두유,콩기름,홍합포함",pesco
...,...,...,...,...,...,...
2920,간식/과자/떡,이브뚜리에,봉봉 초콜렛 12P,"우유,대두,밀","아몬드,천연향료,유청,전화당시럼,해이즐,전분,코코아파우더,글리셰린,프랄린,구연산,전...",general
2921,간식/과자/떡,청우,아홉 번 구워만든 죽염캔디 38g,"땅콩,대두,우유,복숭아","무엇,자일리톨,박하유",vegan
2922,간식/과자/떡,타르투프랑게,트뤼플랏 어소티드 스위트 초콜렛 70g (팩),"밀,우유,대두,보리,땅콩,계란","아몬드,크런치캐러멜,준초콜릿,포도당시련,해이즐,화이트초콜릿,코코아애스,유지방문일유당...",lacto
2923,간식/과자/떡,kurly,맛다시마 젤리 100g (냉장),없음,"설탕,소금,대두,매실과육,다시매,간장",vegan


### 분류 예시

In [14]:
sample_data

,category,brand,item,allergy,ingredient
0,간식/과자/떡,선물세트,창억 프리미엄 선물세트,"호두,잣,밀,계란,대두","설탕,정제소금,산도조절제,호두,변성전분,베이킹파우더,정제수설탕,대추,동팔앙금,천일염..."
1,면/앙념/오일,자연한알,채소육수 40정,"토마토,복숭아,대두,밀,메밀,닭고기,쇠고기,조개류,굴,전복,홍합포함,","설탕,정제소금,마요네즈,외사비분말고추냉이분,아보카도오일,누운마늘향,명란첫길명태알,고..."
2,면/앙념/오일,청정원,소갈비 양념장 2종,"대두,밀,조개류,굴,게,새우,토마토,쇠고기,우유,닭고기,돼지고기",우유
3,음료/우유/커피/차,나탈리스,프리미엄 착즙 오렌지 주스 2종 (택1),"계란,알류,대두,밀,복숭아,우유,토마토","당류,조니"
4,국/반찬/메인 요리,오마뎅,볼 어묵 2종,대두,"설탕,어묵,콜산,폴란드,글루코노델타락톤,소르비톨,보존료,정제수"


In [15]:
sample_data['classify']='0'
for i in range(len(sample_data)):
    flag=0
    milk=0
    egg=0
    for word in sample_data['ingredient'][i].split(','):
        if word in meat_lst:
            flag=1
            break
    if flag==0:
        for word in sample_data['ingredient'][i].split(','):
            if word in sea_lst:
                flag=2
                break
    if flag==0:
        for word in sample_data['ingredient'][i].split(','):
            if word in milk_lst:
                flag=3
                milk+=1
                break
        for word in sample_data['ingredient'][i].split(','):
            if word in egg_lst:
                flag=3
                egg+=1
                break
    if flag==0:
        for word in sample_data['ingredient'][i].split(','):
            if word in honey_lst:
                flag==4
                break
            
    if flag==1:
        classify='general'
    elif flag==2:
        classify='pesco'
    elif flag==3:
        if milk>0 and egg==0:
            classify='lacto'
        elif milk==0 and egg>0:
            classify='ovo'
        else:
            classify='lacto ovo'
    elif flag==4:
        classify='honey'
    else:
        classify='vegan'
    sample_data['classify'][i]=classify
    print('['+sample_data['item'][i]+' ]은 '+sample_data['classify'][i]+'식품입니다.')

[ 창억 프리미엄 선물세트 ]은 ovo식품입니다.
[ 채소육수 40정 ]은 ovo식품입니다.
[ 소갈비 양념장 2종 ]은 lacto식품입니다.
[ 프리미엄 착즙 오렌지 주스 2종 (택1) ]은 vegan식품입니다.
[ 볼 어묵 2종 ]은 pesco식품입니다.


## 추천시스템

In [16]:
train_data['ingredient']=train_data['ingredient'].apply(lambda x: x.replace(',',' '))
count_vect=CountVectorizer(min_df=0,ngram_range=(1,2))
ingredient_mat=count_vect.fit_transform(df['ingredient'])
ingredient_sim=cosine_similarity(ingredient_mat,ingredient_mat)
ingredient_sim_sorted_ind=ingredient_sim.argsort()[:,::-1]

In [17]:
def find_sim_ingredient(df,sorted_ind,product_name,top_n=10,cate=0,vegan=0):
    
    title_product=df[df['item']==product_name]
    title_index=title_product.index.values

    similar_indexes=sorted_ind[title_index]
    similar_indexes=similar_indexes[similar_indexes!=title_index]

    similar_indexes=similar_indexes.reshape(-1)
    option_lst=similar_indexes

    # 카테고리 옵션
    if cate==1:
        option_lst=[]
        category=df.loc[df['item']==product_name,'category'].values
        cate_indexes=df.loc[df['category']==category[0]].index.values
    
    # 비건단계 옵션1 -같은 종류의 단계만 추천
    if vegan==1:
        option_lst=[]
        vegan_level=df.loc[df['item']==product_name,'classify'].values
        same_vegan_indexes=df.loc[df['classify']==vegan_level[0]].index.values

    # 비건단계 옵션2 -비건단계당 2개씩 추천
    elif vegan==2:
        option_lst=[]
        vegan_check={'pesco':0,'lacto ovo':0,'lacto':0,'ovo':0,'vegan':0,'honey':0}
        cnt=0
            
    # 비건단계 옵션3 - 비건 제품만 추천
    elif vegan==3:
        option_lst=[]
        vegan_indexes=df.loc[df['classify']=='vegan'].index.values

    # 옵션 적용
    if cate==1 and vegan==0: # 카테고리
        for idx in similar_indexes:
            if idx in cate_indexes:
                option_lst.append(idx)
    
    elif cate==0 and vegan==1: # 같은비건단계
        for idx in similar_indexes:
            if idx in same_vegan_indexes:
                option_lst.append(idx)

    elif cate==0 and vegan==2: # 비건단계별
        for idx in similar_indexes:
            vegan_level=df.iloc[idx,-1]
            if vegan_level!='general' and vegan_check[vegan_level]<2:
                vegan_check[vegan_level]+=1
                option_lst.append(idx)
                cnt+=1
            elif cnt==12:
                break
            else:
                continue
    
    elif cate==0 and vegan==3: # 비건제품만
        for idx in similar_indexes:
            if idx in vegan_indexes:
                option_lst.append(idx)

    elif cate==1 and vegan==1: # 카테고리, 같은비건단계
        for idx in similar_indexes:
            if idx in cate_indexes and idx in same_vegan_indexes:
                option_lst.append(idx)

    elif cate==1 and vegan==2: # 카테고리, 비건단계별
        for idx in similar_indexes:
            if idx in cate_indexes:
                vegan_level=df.iloc[idx,-1]
                if vegan_level!='general' and vegan_check[vegan_level]<2:
                    vegan_check[vegan_level]+=1
                    option_lst.append(idx)
                    cnt+=1
                elif cnt==12:
                    break
                else:
                    continue

    elif cate==1 and vegan==3:  # 카테고리, 비건제품만
        for idx in similar_indexes:
            if idx in cate_indexes and idx in vegan_indexes:
                option_lst.append(idx)

    return df.iloc[option_lst][:top_n].sort_values('classify')

### 추천시스템 예시

In [18]:
sample_data['ingredient']=sample_data['ingredient'].apply(lambda x:x.replace(',',' '))

In [19]:
# 옵션 입력
print('카테고리 옵션을 선택하세요')
print('1) 같은 카테고리 식품만 궁금해요')
print('2) 다른 카테고리 식품도 궁금해요')
cate_option=input('1 / 2')

print('=> '+cate_option+'번 옵션 선택')

if cate_option=='1':
    cate_option=1
else:
    cate_option=0

print('\n비건 옵션을 선택하세요')
print('1) 비건 단계 상관없이 모두 궁금해요')
print('2) 같은 비건 단계 식품만 궁금해요')
print('3) 다른 비건단계 식품들도 궁금해요')
print('4) 비건인 식품만 궁금해요')
vegan_option=input('1 / 2 / 3 / 4')

print('=> '+vegan_option+'번 옵션 선택')

if vegan_option=='1':
    vegan_option=0
elif vegan_option=='2':
    vegan_option=1
elif vegan_option=='3':
    vegan_option=2
else:
    vegan_option=3

카테고리 옵션을 선택하세요
1) 같은 카테고리 식품만 궁금해요
2) 다른 카테고리 식품도 궁금해요
=> 1번 옵션 선택

비건 옵션을 선택하세요
1) 비건 단계 상관없이 모두 궁금해요
2) 같은 비건 단계 식품만 궁금해요
3) 다른 비건단계 식품들도 궁금해요
4) 비건인 식품만 궁금해요
=> 2번 옵션 선택


In [20]:
input_data=sample_data.loc[[0]]
input_data

,category,brand,item,allergy,ingredient,classify
0,간식/과자/떡,선물세트,창억 프리미엄 선물세트,"호두,잣,밀,계란,대두",설탕 정제소금 산도조절제 호두 변성전분 베이킹파우더 정제수설탕 대추 동팔앙금 천일염...,ovo


In [21]:
df=pd.concat([input_data,train_data]).reset_index(drop=True)
ingredient_mat=count_vect.fit_transform(df['ingredient'])
ingredient_sim=cosine_similarity(ingredient_mat,ingredient_mat)
ingredient_sim_sorted_ind=ingredient_sim.argsort()[:,::-1]

print('입력한 상품과 비슷한 제품을 보여드려요')
find_sim_ingredient(df,ingredient_sim_sorted_ind,input_data['item'].values[0],10,cate_option,vegan_option)

입력한 상품과 비슷한 제품을 보여드려요


,category,brand,item,allergy,ingredient,classify
2587,간식/과자/떡,창억,창억 모듬 떡케이크,"계란,밀,잣,대두,호두",설탕 천일염 일가로 가공소금 계란 정제수,ovo
2586,간식/과자/떡,창억,호박인절미 프리미엄 선물세트,"계란,밀,대두,호두,잣",떡류 정제수설탕 웰가어서피 한발가루 가공소금 찹쌀 계란 정제수,ovo
2231,간식/과자/떡,kurly,타케다 오리미소 센베이,"대두,밀,계란,땅콩,우유,캐슈넛",설탕 정제소금 과자 대두 된장 소맥분 계란,ovo
2459,간식/과자/떡,경기떡집,이티떡,"우유,계란,밀,땅콩,대두,호두,잣",호두 설탕 대두 참기름 계란,ovo
2390,간식/과자/떡,상하목장,얼려먹는 아이스주스 망고,"우유,대두,땅콩,밀,계란,토마토,복숭아,호두,닭고기",호두 대두 땅콩 토마토 복숭아 터키산 유기농 계란 정제수,ovo
2688,간식/과자/떡,해마루,파주 장단콩 팥소빵 (5개입),"계란,밀,대두",설탕 밀가루 산도조절제 산도조절제전분 변성전분 앗술 베이킹파우더 대두 양조간장 방류...,ovo
2167,간식/과자/떡,착한마을 마음이가,현미 귀리 가래떡,"대두,계란,우유,땅콩,호두,밀",천일염 귀리 계란,ovo
2326,간식/과자/떡,고디바,펄 초콜릿 3종,"우유,대두",호두 땅콩 계란,ovo
2313,간식/과자/떡,착한떡,제수용 콩시루떡,"대두,땅콩,호두,밀,우유,계란,잣,아황산류,오징어,복숭아",백설탕 호두 소금 떡류 대두 땅콩 아황산류 천일염 계란,ovo
2813,간식/과자/떡,이장님과자점,우리콩 참두부과자 5개입,"밀,대두,계란,우유,땅콩,아황산류,호두,잣",밀가루 설탕 대두유 소금 호두 당류 과자 대두 아황산류 유탕처리제품 염화마그네슴 두...,ovo


## 알레르기

### 알레르기 예시

In [22]:
user_allergy=[]
user_input=''
print('알레르기 정보를 입력하세요.\n다 입력하신 후 \"끝\"을 입력해주세요.')
while user_input!='끝':
    user_input=input()
    print(user_input)
    user_allergy.append(user_input)
user_allergy=user_allergy[:-1]

알레르기 정보를 입력하세요.
다 입력하신 후 "끝"을 입력해주세요.
1
1
1
1
1
1
1
1
끝


In [23]:
for i in range(len(sample_data)):
    allergy_lst=[]
    for word in sample_data['allergy'][i].split(','):
        if word in user_allergy:
            allergy_lst.append(word)
    if len(allergy_lst)>0:
        print('['+sample_data['item'][i]+']에는 고객님이 입력하신 알레르기 중 '+str(allergy_lst)+'가 존재합니다. 조심하세요!')
    else:
        print('['+sample_data['item'][i]+']에는 고객님이 입력하신 알레르기가 없어요. 안심하세요!')

[ 창억 프리미엄 선물세트]에는 고객님이 입력하신 알레르기가 없어요. 안심하세요!
[ 채소육수 40정]에는 고객님이 입력하신 알레르기가 없어요. 안심하세요!
[ 소갈비 양념장 2종]에는 고객님이 입력하신 알레르기가 없어요. 안심하세요!
[ 프리미엄 착즙 오렌지 주스 2종 (택1)]에는 고객님이 입력하신 알레르기가 없어요. 안심하세요!
[ 볼 어묵 2종]에는 고객님이 입력하신 알레르기가 없어요. 안심하세요!


### 글루텐 예시

In [24]:
glu=['글루테닌','글리아딘','보리','귀리','밀','박력분','강력분','중력분']
for i in range(len(sample_data)):
    flag=0
    for word in sample_data['ingredient'][i].split(','):
        if word in glu:
            print(word)
            flag=1
            break
    if flag==1:
        print('['+sample_data['item'][i]+' ]에는 글루텐이 포함되어 있습니다.')
    else:
        print('['+sample_data['item'][i]+' ]에는 글루텐이 포함되어 있지 않습니다.')

[ 창억 프리미엄 선물세트 ]에는 글루텐이 포함되어 있지 않습니다.
[ 채소육수 40정 ]에는 글루텐이 포함되어 있지 않습니다.
[ 소갈비 양념장 2종 ]에는 글루텐이 포함되어 있지 않습니다.
[ 프리미엄 착즙 오렌지 주스 2종 (택1) ]에는 글루텐이 포함되어 있지 않습니다.
[ 볼 어묵 2종 ]에는 글루텐이 포함되어 있지 않습니다.
